In [1]:
import argparse
import networkx as nx
import tensorflow as tf # TODO use gpu
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()
import my

/home/gaiyu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
args = argparse.Namespace()
args.depth = 10
args.device = '/cpu:0'
args.graph = 'soc-Epinions1-reduced'
args.n_features = 8
args.n_machines = 10
args.radius = 3

In [3]:
class GNNModule(tf.keras.Model):
    def __init__(self, in_features, out_features, adj, nonlinear):
        super().__init__()
        self.adj = adj
        self.deg = tf.reduce_sum(adj[0], 1, keepdims=True)
        new_linear = lambda: nn.Parameter(th.randn(in_features, out_features / 2))
        self.alpha1, self.alpha2, self.alpha3 = new_linear(), new_linear(), new_linear()
        self.alpha4 = nn.ParameterList([new_linear() for a in adj])
        self.beta1, self.beta2, self.beta3 = new_linear(), new_linear(), new_linear()
        self.beta4 = nn.ParameterList([new_linear() for a in adj])
        self.bn_alpha, self.bn_beta = nn.BatchNorm1d(out_features), nn.BatchNorm1d(out_features)
        self.nonlinear = nonlinear
    
    def forward(self, x):
        deg = self.deg * x
        u = th.zeros_like(x) + th.mean(x, 1, keepdim=True)
        adj = [th.mm(a, x) for a in self.adj]
        alpha = th.mm(x, self.alpha1) + th.mm(deg, self.alpha1) + th.mm(u, self.alpha2) + \
            sum(th.mm(a, alpha) for alpha, a in zip(self.alpha4, adj))
        alpha = self.bn_alpha(self.nonlinear(alpha))
        beta = th.mm(x, self.beta1) + th.mm(deg, self.beta1) + th.mm(u, self.beta2) + \
            sum(th.mm(a, beta) for beta, a in zip(self.beta4, adj))
        beta = self.bn_beta(beta)
        return th.cat((alpha, beta), 1)

class EdgeDense(tf.keras.Model):
    def __init__(self, in_features, out_features, adj):
        super().__init__()
        self.adj = adj
        self.out_features = out_features
        for i in range(out_features):
            setattr(self, 'dense%d' % i, tf.keras.layers.Dense(input_shape=(in_features,), units=1))
    
    def call(self, x):
        z_list = []
        for i in range(self.out_features):
            z = getattr(self, 'dense%d' % i)(x)
            z = tf.sparse_add(z * self.adj, tf.transpose(z) * self.adj)
            z_list.append(tf.reshape(z.values, (-1, 1)))
        z = tf.concat(z_list, 1)
        return z

class GNN(tf.keras.Model):
    def __init__(self, features, n_classes, adj, radius, nonlinear, dense):
        super().__init__()
        adj = my.sparse_sp2th(adj).float()
        if dense:
            adj = adj.to_dense()
        a, adj_list = adj, [adj]
        for i in range(radius - 1):
            a = th.mm(a, a)
            adj_list.append(a)
        self.module_list = nn.ModuleList([GNNModule(m, n, adj_list, nonlinear)
                                          for m, n in zip(features[:-1], features[1:])])
        self.linear = EdgeLinear(features[-1], n_classes, adj)
    
    def forward(self, x):
        for module in self.module_list:
            x = module(x)
        x = self.linear(x)
        x = F.softmax(x, 1)
        return x

In [4]:
class Objective:
    def __init__(self, adj, n_machines):
        n = tf.cast(adj.dense_shape[0], tf.int32)
        indices = tf.cast(adj.indices, tf.int32)
        s = tf.one_hot(indices[:, 0], n) + tf.one_hot(indices[:, 1], n)
        self.s = tf.transpose(s)
        self.n_machines = n_machines
        
    def __call__(self, x):
        y = tf.multinomial(x, num_samples=1)
        y = tf.squeeze(y)
        y = tf.one_hot(y, self.n_machines)
        y = tf.matmul(self.s, y)
        r = tf.reduce_sum(y)
        p = (tf.reduce_sum(y, 1) + 1) / r
        b = -tf.reduce_sum(p * tf.log(p))
        objective = -(r + b) * tf.log(x)
        return objective

In [5]:
g = my.read_edgelist(args.graph)

In [6]:
with tf.device(args.device):
# with tf.device('/device:GPU:0'):
    adj = tf.cast(my.sparse_sp2tf(nx.adj_matrix(g)), tf.float32)
    objective = Objective(adj, args.n_machines)
    dense = EdgeDense(args.n_features, args.n_machines, adj)
    optimizer = tf.train.AdamOptimizer()
    
    x = tfe.Variable(tf.random_normal((adj.dense_shape[0], args.n_features)))

In [7]:
with tf.device(args.device):
    with tfe.GradientTape() as tape:
        x = objective(dense(x))

    gradients = tape.gradient(x, dense.variables)
    optimizer.apply_gradients(zip(gradients, dense.variables)) # TODO tf.train.get_or_create_global_step